Skaffold.dev
==========

![](https://skaffold.dev/images/architecture.png)

Quelle: https://skaffold.dev/docs/design/

- - - 

[Skaffold](https://skaffold.dev) ist ein Open-Source-Tool von Google, das den Entwicklungsprozess für Kubernetes-native Anwendungen vereinfacht. Es übernimmt den gesamten Workflow für das Erstellen, Pushen und Bereitstellen deiner Anwendung, sodass du dich auf das Wesentliche konzentrieren kannst: das Schreiben von Code.

Mit Skaffold kannst du deine Projekte schnell und wiederholbar entwickeln und bereitstellen, ohne dass eine aufwendige Konfiguration erforderlich ist. Es unterstützt verschiedene Entwicklungsumgebungen und bietet zahlreiche Funktionen wie policy-basierte Image-Tagging, Port-Forwarding, Logging und Dateisynchronisation.

- - -

Um zu beginnen, müssen wir zuerst die Container Image Registry in microk8s enablen:

In [ ]:
! microk8s enable registry

Kontrollieren ob alles sauber gestartet wurde:

In [ ]:
! kubectl get pods,service --namespace container-registry

Anschliessend stellen wir die Software zusammen.

* [autoshop-ms](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop) - im Verzeichnis `shop`
* [Kubernetes Deklarationen](https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates) im Verzeichnis `shop/k8s` 

In [ ]:
%%bash
# Clean
sudo rm -rf shop/ kubernetes-templates/
# Sourcecode
git clone --branch v2.1.0 https://gitlab.com/ch-mc-b/autoshop-ms/app/shop.git
# Kubernetes Deklarationen
git clone https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates.git
cp -r kubernetes-templates/2.1.0-deployment/ shop/k8s/
cp kubernetes-templates/*-service.yaml kubernetes-templates/*-ingress.yaml shop/k8s/
# Registry von Gitlab auf lokale Registry umstellen
for yaml in shop/k8s/*-deployment.yaml
do
    sed -i -e 's;registry.gitlab.com/ch-mc-b/autoshop-ms/app/;localhost:32000/;g' ${yaml}
done
sudo rm -rf kubernetes-templates shop/.git shop/docker-compose.yml shop/m1
ls -l shop
cat shop/k8s/order-deployment.yaml

### Skaffold

Als letztes bereiten wie die Deklaration für Skaffold vor.

Diese erstellt automatisch 
* die Container Images, mit `docker`
* legt diese in der lokalen Container Registry `localhost:32000` ab
* deployt die Container Images nach Kubernetes

In [ ]:
%%bash
cat <<EOF >shop/skaffold.yaml
apiVersion: skaffold/v4beta11
kind: Config
metadata:
  name: shop-skaffold
build:
  artifacts:
    - image: localhost:32000/shop/catalog
      context: catalog
      docker:
        dockerfile: Dockerfile
    - image: localhost:32000/shop/customer
      context: customer
      docker:
        dockerfile: Dockerfile
    - image: localhost:32000/shop/order
      context: order
      docker:
        dockerfile: Dockerfile
    - image: localhost:32000/shop/shop
      context: webshop
      docker:
        dockerfile: Dockerfile
manifests:
  rawYaml:
    - k8s/catalog-deployment.yaml
    - k8s/catalog-service.yaml
    - k8s/customer-deployment.yaml
    - k8s/customer-service.yaml
    - k8s/order-deployment.yaml
    - k8s/order-service.yaml
    - k8s/webshop-deployment.yaml
    - k8s/webshop-ingress.yaml
    - k8s/webshop-service.yaml
EOF


Zuerst lassen wir `skaffold` die Container Images bauen.

Diese stehen dann als Container Images in Docker und der Registry (leider nicht durchsuchbar) zur Verfügung:

In [ ]:
%%bash
cd shop; skaffold build --namespace=shop-skaffold
docker image ls | grep localhost:32000

Anschliessend Deployen wir die Container Images in Kubernetes

In [ ]:
%%bash
kubectl create namespace shop-skaffold
cd shop; skaffold run --namespace=shop-skaffold
kubectl get pods,services --namespace=shop-skaffold
echo "http://"$(cat ~/work/server-ip)":$(kubectl get service webshop --namespace=shop-skaffold -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop"

- - -

### Aufräumen

In [ ]:
%%bash
cd shop; skaffold delete --namespace=shop-skaffold
kubectl delete namespace shop-skaffold